### Word2Vec을 통한 임베딩 진행 

https://doitgrow.com/27

python3 -m pip install konlpy

pip install beautifulsoup4

pip install --upgrade gensim

pip install soynlp

apt-get update

apt install default-jdk

pip install git+https://github.com/haven-jeon/PyKoSpacing.git

pip install git+https://github.com/ssut/py-hanspell.git

load및 설치해야 하는 대상이 상당히 많습니다.

위에 친구들을 모두 설치해주어야 합니다.

In [3]:
# 필요한 객체 import
import pandas as pd
from konlpy.tag import Okt
from preprocessing import preprocess_data, tokenizing_data, get_label
from utils import seed_fix
import pandas as pd
import numpy as np
import random
import pickle

import re
# 초기화 및 모델 학습
from gensim.models import word2vec
# 띄어쓰기
from pykospacing import Spacing
# 마춤뻡 검사기
from hanspell import spell_checker

from soynlp.normalizer import *

바로 아래 코드는 직접 word2vec을 만들어줄 때만 필요한 contexts를 만들어주는 파일입니다.

이 부분은 넘어가셔도 됩니다.

In [ ]:
df1 = pd.read_csv("../data/" + "KorSTS/sts-train.tsv", sep='\t')
df2 = pd.read_csv("../data/" + "KorSTS/sts-dev.tsv", sep='\t')
df3 = pd.read_csv("../data/" + "KorSTS/sts-test.tsv", sep='\t')
df4 = pd.read_csv("../data/" + "whole_answers.csv", sep=',', encoding = 'utf-8')
data = open("../data/para_kqc_sim_data.txt", 'r', encoding='utf-8')
lines = data.readlines()
random.shuffle(lines)
train_data, test_data = preprocess_data(lines)

seed_fix(42)


train_contexts1 = list(
    df1.iloc[i].sentence1 for i in range(len(df1))
    )  # set 은 매번 순서가 바뀌므로
train_contexts2 = list(df1.iloc[j].sentence2 for j in range(len(df1)))
dev_contexts1 = list(
    df2.iloc[i].sentence1 for i in range(len(df2))
        )  # set 은 매번 순서가 바뀌므로
dev_contexts2 = list(df2.iloc[j].sentence2 for j in range(len(df2)))
test_contexts1 = list(
    df3.iloc[i].sentence1 for i in range(len(df3))
    )
test_contexts2 = list(df3.iloc[j].sentence2 for j in range(len(df3)))

parpara_contexts1 = list(
    train_data.iloc[i].sent_a for i in range(len(train_data))
    )
parpara_contexts2 = list(train_data.iloc[j].sent_b for j in range(len(train_data)))

parpara_contexts3 = list(
    test_data.iloc[i].sent_a for i in range(len(test_data))
    )
parpara_contexts4 = list(test_data.iloc[j].sent_b for j in range(len(test_data)))
wai_contexts = list(
    df4["answeres"].iloc[i] for i in range(len(df4))
    )


contexts = train_contexts1 + train_contexts2 + dev_contexts1 + dev_contexts2 + test_contexts1 + test_contexts2 + parpara_contexts1 + parpara_contexts2 + parpara_contexts3 + parpara_contexts4 + wai_contexts

import pickle

with open("contexts.pkl", "wb") as f:
    pickle.dump(contexts, f)

with open('contexts.pkl','rb') as f:
    contexts = pickle.load(f)

가장 먼저 Okt를 불러와서 형태소 분석을 해줄 tokenize 함수를 선언해줍니다.

해당 함수는 단어에 대해서 Noun, Eomi(어미), Josa(조사), Verb 등 태깅을 해줌과 동시에 형태소 분할도 진행해줍니다.

In [51]:
pos_tagger = Okt()

def tokenize(doc):
    # norm, stem은 optional
    if type(doc) is not str:
        return []
    # return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
    return [t[0] for t in pos_tagger.pos(doc, norm=True, stem=True)]

tokenized_list = []

for i in range(len(contexts)):    
    review = tokenize(contexts[i])
    tokenized_list.append(review) # 토큰화 된 리뷰를 리스트에 담아줌
    
print(tokenized_list[:10])

[['비행기', '가', '이륙', '하고', '있다'], ['하다', '남자', '가', '크다', '플루트', '를', '연주', '하고', '있다'], ['하다', '남자', '가', '피자', '에', '치즈', '를', '뿌리다', '있다'], ['세', '남자', '가', '체스', '를', '하다', '있다'], ['하다', '남자', '가', '첼로', '를', '연주', '하고', '있다'], ['몇몇', '남자', '들', '이', '싸우다', '있다'], ['남자', '가', '담배', '를', '피우다', '있다'], ['남자', '가', '피아노', '를', '치고', '있다'], ['하다', '남자', '가', '기타', '를', '치고', '노래', '를', '부르다', '있다'], ['사람', '이', '고양이', '를', '천장', '에', '던지다', '있다']]


아래의 방식을 통해서 직접 Word2vec을 구현해줄 수 있습니다.

하지만 OOV가 많이 등장하고 성능이 높지 못해 프로토타입으로 남겨두었습니다.

In [52]:
#하이퍼 파라메터 값 지정
num_features = 400 # 문자 벡터 차원수
min_word_count = 5 # 최소 문자 수
num_workers = 4 # 병렬 쓰레드 수
context = 10 # window size
downsampling = 1e-3 # 문자 빈도 수 downsample

#모델 학습
model = word2vec.Word2Vec(
                            tokenized_list,
                            # double_contexts,
                            workers = num_workers,
                            vector_size = num_features,
                            min_count = min_word_count,
                            window = context,
                            sample = downsampling
                        )



In [53]:
# 학습이 완료 되면 필요없는 메모리를 unload 시킨다.
model.init_sims(replace=True)

model_name = 'Word2Vec_Kor_POS_Tagging_WAI'
# model_name = '300features_50minwords_20text'
model.save(model_name)
model.wv.save_word2vec_format('my_pos_tagging_wai.embedding', binary=False)

/tmp/ipykernel_31436/3004066225.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


### 여기서부터 진짜

가장 먼저 유사도를 구해줄 cosine similarity 함수를 선언해줍니다.

행렬이 아닌 벡터이기 때문에 sklearn 활용이 불가능해 직접 만들어주었습니다.

In [4]:
def cosine_similarity(a, b):

    #return np.inner(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

아래 클래스를 통해서 유사도를 계산해줍니다.

3중 for문을 돌아야 하는 것을 함수로 넘가도록 구현했습니다.

1. 키워드 리스트(키워드 여러 개) + 문장 리스트(답안 여러개)
2. 키워드 하나 + 문장 리스트
3. 키워드 하나 + 문장 하나

이런 식으로 해서 차례대로 값을 반환해주는 방식입니다.

현재는 이 때문에 3중 리스트형식으로 구현이 되어 있습니다.

In [5]:
import re

class Keyword_similarity:
    
    def __init__(self, model, threshold = 0.5, pos_tagger=None, lemmatizer=None):
        self.model = model
        self.threshold = threshold
        self.pos_tagger = pos_tagger
        self.lemmatizer = lemmatizer
    
    ### 키워드랑 문장이 리스트로 들어온다.

    def keyword_score(self, keyword_list, sentence_list):
        idx_list = []
        for keyword in keyword_list:
            idx_list.append(self.get_keyword_score_list(keyword, sentence_list))

        return idx_list

    ## 키워드 하나에 대해서 답안들에 대해 조사한다.
    def get_keyword_score_list(self, keyword, sentence_list):
        idx_list = []
        for sentence in sentence_list:
            idx_list.append(self.keyword_one_sentence(keyword, sentence))
        
        return idx_list

    ## 하나의 키워드에 대해서 그 문장의 것들에 대한 점수를 리스트로 반환한다.
    def keyword_one_sentence(self, keyword, sentence):
        # pos_keyword = self.tokenize(keyword)
        # keyword_vec = self.model.wv.get_vector(pos_keyword[0])
        pos_keyword = pos_tagger.pos(keyword)
        keyword_vec = self.model.wv.get_vector(pos_keyword[0][0])
        
        cosine_list = []
        word_list = []
        split_sentence = sentence.split()
        for word in split_sentence:
            
            pos_word = pos_tagger.pos(word)            
            # pos_word = word
            for split_word in pos_word:             
                if split_word[1] in ['Noun', 'Verb', 'Adjective', 'Adverb']:
                # if split_word[1] in ['Noun']:
                    try: 
                        word_vec = self.model.wv.get_vector(split_word[0])
                        # word_vec = self.model.wv.get_vector(pos_word)
                        # word_vec = self.model.wv.get_vector(pos_word)
                    except:
                        continue
                    if cosine_similarity(keyword_vec, word_vec) > self.threshold and pos_word not in word_list:
                        results = re.finditer(word, sentence)
                        for matched_key in results:
                            cosine_list.append([matched_key.start(), matched_key.end()])
                        
                        word_list.append(pos_word[0][0])
            
        return cosine_list, word_list

In [23]:
class Keyword_similarity:
    
    def __init__(self, model, threshold = 0.5, pos_tagger=None, lemmatizer=None):
        self.model = model
        self.threshold = threshold
        self.pos_tagger = pos_tagger
        self.lemmatizer = lemmatizer
    
    ### 키워드랑 문장이 리스트로 들어온다.
    def tokenize(self, doc):
    # norm, stem은 optional
        if type(doc) is not str:
            return []
        return ['/'.join(t) for t in self.pos_tagger.pos(doc, norm=True, stem=True)]
    
    def get_keyword_score(self, keyword_list, sentence_list):
        results = self.keywords_sentences(keyword_list, sentence_list)
        
        keyword_num = len(results.keys())
        sentence_num = len(results.values())
        
        keyword_score = []
        for keyword, students in results.items():
            for idx, sentence in students.items():    
                keyword_score.append(len(sentence[0]))
        return results, keyword_score
        

    def keywords_sentences(self, keyword_list, sentence_list):
        keyword_dict = {}
        for idx, keyword in enumerate(keyword_list):
            keyword_dict[f"keyword_{idx}"] = self.keyword_sentences(keyword, sentence_list)

        return keyword_dict

    ## 키워드 하나에 대해서 답안들에 대해 조사한다.
    def keyword_sentences(self, keyword, sentence_list):
        sentence_dict = {}
        for idx, sentence in enumerate(sentence_list):
            sentence_dict[f'student_{idx}'] = self.keyword_one_sentence(keyword, sentence)
        
        return sentence_dict

    ## 하나의 키워드에 대해서 그 문장의 것들에 대한 점수를 리스트로 반환한다.
    def keyword_one_sentence(self, keyword, sentence):
        # pos_keyword = self.tokenize(keyword)
        # keyword_vec = self.model.wv.get_vector(pos_keyword[0])
        pos_keyword = pos_tagger.pos(keyword)
        keyword_vec = self.model.wv.get_vector(pos_keyword[0][0])
        
        start_idx = []
        end_idx = []
        word_list = []
        split_sentence = sentence.split()
        for word in split_sentence:
            
            pos_word = pos_tagger.pos(word)            
            # pos_word = word
            for split_word in pos_word:             
                if split_word[1] in ['Noun', 'Verb', 'Adjective', 'Adverb']:
                # if split_word[1] in ['Noun']:
                    try: 
                        word_vec = self.model.wv.get_vector(split_word[0])
                        # word_vec = self.model.wv.get_vector(pos_word)
                        # word_vec = self.model.wv.get_vector(pos_word)
                    except:
                        continue
                    if cosine_similarity(keyword_vec, word_vec) > self.threshold and pos_word not in word_list:
                        results = re.finditer(word, sentence)
                        for matched_key in results:
                            start_idx.append(matched_key.start())
                            end_idx.append(matched_key.end())
                        
                        word_list.append(pos_word[0][0])
            
        return start_idx, end_idx, word_list

fastText로 다운 받은 모델을 로드합니다

In [6]:
import gensim 
import gensim.models as g

model_name = 'fast_text_ko'
model = g.Doc2Vec.load(model_name)

아래 방식은 gz 파일 자체로 로드하는 방식이어서 9분 정도 걸립니다(비추)

In [ ]:
# from gensim import models

# ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin.gz')

모델 저장 코드

In [ ]:
# ko_model.save("fast_text_ko")
# ko_model.wv.save_word2vec_format('fast_text.embedding', binary=False)

형태소 분석기와 word2vec load 후 단일 문장에서 제대로 작동하는지 확인

In [25]:
pos_tagger = Okt()
Fast_KS = Keyword_similarity(model, 0.35, pos_tagger)
print(Fast_KS.keyword_one_sentence("가격", "어느 곳에 빵의 값이 낮은 지 알 수 있다."))
print(Fast_KS.keyword_one_sentence("품질", "어느 곳에 맛이 좋은 지 알 수 있다."))
print(Fast_KS.keyword_one_sentence("압력", "기압이 낮아지게 되어 부피가 커진다"))

([9], [11], ['값'])
([6, 9], [8, 11], ['맛', '좋은'])
([0, 12], [3, 15], ['기압', '부피'])


여러 키워드, 여러 문장 가능한지 확인

In [47]:
print(Fast_KS.keywords_sentences(["가격", "품질"], ["어느 곳에 빵의 값이 저렴한 지 알 수 있다.", "어느 곳에 맛이 좋은 지 알 수 있다.", "어느 곳에 가격이 낮은 지 알 수 있다."]))
result = Fast_KS.keywords_sentences(["가격", "품질"], ["어느 곳에 빵의 값이 저렴한 지 알 수 있다.", "어느 곳에 맛이 좋은 지 알 수 있다.", "어느 곳에 가격이 낮은 지 알 수 있다."])

{'keyword_0': {'student_0': ([9, 12], [11, 15], ['값', '저렴한']), 'student_1': ([], [], []), 'student_2': ([6], [9], ['가격'])}, 'keyword_1': {'student_0': ([], [], []), 'student_1': ([6, 9], [8, 11], ['맛', '좋은']), 'student_2': ([6], [9], ['가격'])}}


In [52]:
keyword_num = len(result.keys())
sentence_num = len(result.values())
corrected = []
for keyword, students in result.items():
    temp = []
    for idx, sentence in students.items():    
        if len(sentence[0]) != 0:
            temp.append(1)
        else : 
            temp.append(0)

    corrected.append(temp)

keyword_score = [0 for i in range(sentence_num)]
for score in corrected:
    for i in range(sentence_num):
        keyword_score[i] += score[i]

AttributeError: 'dict_values' object has no attribute 'keys'

In [44]:
print(keyword_score)

[1, 1]
